# Consumer Memory Agent
**Applying a multimodal thread of reasoning to customer service**     
copyright 2025, Denis Rothman

The goal of this notebook is to develop a consumer memory agent that applies neuroscience-inspired memory tagging to customer reviews, enabling the generation of personalized messages.

**Table of Contents**

1. **Setting up the environment**

2. **The dataset: hotel reviews**

3. **Agent Memory Functions**
   - Querying Agent Memory
   - Upserting Functions
   - Memory Structure

4. **Building the Agent's Memory Structure**
    - Memory Hierarchy
    - Visualizing Memory Categories

5. **Strategic consumer agent memory**
    - Hotel Reviews Analysis
    - Sentiment Analysis and Memory Tagging
    - Creating Marketing Content
    - Image and Customer Message Generation

**Note**: *This notebook is for educational purposes only. It is not designed to be deployed into production.*

This notebook uses OpenAI GPT Models. https://openai.com




# Setting up the environment

This notebook was developed in Google Colab. Colab includes many pre-installed libraries and sets `/content/` as the default directory, meaning you can access files directly by their filename if you wish (e.g., `filename` instead of needing to specify `/content/filename`). This differs from local environments, where you'll often need to install libraries or specify full file paths.

## Installing `click` and gTTS

**If you want to use text-to-speech in this notebook with `gTTS`, set `use_gtts` to `True` in the following cell.**

`click`, a command line library, is required for gTTS. This cell will set up the correct 'click' version required for gTTS and *display a restart message*. After restarting, simply click `Run All` to continue.

*The restart is necessary for Colab to take the new version into account.*



In [ ]:
use_gtts = False #activates Google TTS in Google Colab if True and deactivates if False

In [ ]:
# Cell 1: Conditional 'click' setup.
# This cell will only modify 'click' IF the 'click' version is not 8.1.8.
# If changes are made, a clear message will appear in the output, instructing you to restart the session.
# After restarting, Colab will reconnect, and you can simply "Run All" from the top.

import importlib.metadata
import os
import time

# Import for displaying rich output (HTML)
from IPython.display import display, HTML

#---Begin gTTS True or
current_click_version = None
try:
    current_click_version = importlib.metadata.version('click')
except importlib.metadata.PackageNotFoundError:
    pass

if not use_gtts:
    required_click_version = current_click_version
else:
    required_click_version = '8.1.8'
#---END gTTS True or False
print("--- Checking 'click' package version... ---")

try:
    current_click_version = importlib.metadata.version('click')
    print(f"Currently installed 'click' version: {current_click_version}")
except importlib.metadata.PackageNotFoundError:
    print("'click' package not found.")

# Check if current click version is not the required one
if current_click_version != required_click_version:
    print(f"\n--- 'click' version is not {required_click_version}. Initiating setup... ---")

    print("Uninstalling any existing 'click' installation...")
    !pip uninstall -y click

    print(f"Installing 'click=={required_click_version}' for compatibility...")
    !pip install click=={required_click_version}

    print("\n--- 'click' setup complete. ---")

    # Display a prominent, styled HTML message in the cell output
    html_message = """
    <div style="
        background-color: #e8f5e9; /* Very light green, calming */
        border: 2px solid #4caf50; /* Green border */
        padding: 15px;
        margin: 15px 0;
        border-radius: 8px;
        font-size: 1.1em; /* Slightly smaller font */
        color: #000000; /* Black text */
    ">
        <h3>Runtime Setup Complete</h3>
        <p>Package versions have been updated. For changes to take effect, you <strong>MUST restart the Colab runtime now.</strong></p>
        <p>Go to the menu: <strong><code>Runtime</code> &gt; <code>Restart runtime</code></strong>.</p>
        <p>After reconnecting, simply click <strong><code>Run All</code></strong> from the top to continue.</p>
    </div>
    """
    display(HTML(html_message))

    # Stop the Python cell execution gracefully.
    raise SystemExit("Please restart the Colab runtime to apply changes.")

else:
    print(f"--- 'click' is already at the correct version ({required_click_version}). No action needed. ---")

## File downloading script

grequests contains a script to download files from the repository

In [ ]:
!curl -L https://raw.githubusercontent.com/uCertifyLLC/Building-Business-Ready-Generative-AI-Systems/main/commons/grequests.py --output grequests.py

## OpenAI

In [ ]:
from grequests import download
download("commons","requirements01.py")
download("commons","openai_setup.py")
download("commons","reason.py")
download("commons","machine_learning.py")

### Installing OpenAI

In [ ]:
# Run the setup script to install and import dependencies
%run requirements01

#### Initializing the OpenAI API key



In [ ]:
google_secrets=True #activates Google secrets in Google Colab
if google_secrets==True:
  import openai_setup
  openai_setup.initialize_openai_api()

In [ ]:
if google_secrets==False: # Uncomment the code and choose any method you wish to initialize the API_KEY
  import os
  #API_KEY=[YOUR API_KEY]
  #os.environ['OPENAI_API_KEY'] = API_KEY
  #openai.api_key = os.getenv("OPENAI_API_KEY")
  #print("OpenAI API key initialized successfully.")

#### Importing the API call function

In [ ]:
# Import the function from the custom OpenAI API file
import os
import reason
from reason import make_openai_api_call

## Installing gtts

gTTS (Google Text-to-Speech) is a Python library and CLI tool that interfaces with Google Translate's text-to-speech API. It allows users to convert text into spoken words, supporting multiple languages and accents, and can save the output as MP3 files.  

In [ ]:
# use_gtts activates Google TTS in Google Colab if use_gtts is True and deactivates if False
if use_gtts:
 !pip install gTTS==2.5.4
 from gtts import gTTS
 from IPython.display import Audio

def text_to_speech(text):
    # Convert text to speech and save as an MP3 file
    if use_gtts:
      if not isinstance(text, str):
            text = str(text) # Making sure the text is a string not a list
      tts = gTTS(text)
      tts.save("response.mp3")

## Machine learning

In [ ]:
# Import the function from the custom OpenAI API file
import os
import machine_learning
from machine_learning import ml_agent

## Web search

In [ ]:
download("commons","web_search.py")

In [ ]:
# Import the function from the custom OpenAI API file
import os
import web_search
from web_search import search

## Chain of Thought(COT)

In [ ]:
# Import the function from the custom OpenAI API file
import os
import reason
from reason import chain_of_thought_reasoning

## Installing Pinecone

In [ ]:
download("commons","requirements02.py")

In [ ]:
# Run the setup script to install and import dependencies
%run requirements02

### Initializing the Pinecone API key

In [ ]:
download("commons","pinecone_setup.py")

In [ ]:
if google_secrets==True:
  import pinecone_setup
  pinecone_setup.initialize_pinecone_api()

In [ ]:
if google_secrets==False: # Uncomment the code and choose any method you wish to initialize the Pinecone API key
  import os
  #PINECONE_API_KEY=[YOUR PINECONE_API_KEY]
  #os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
  #openai.api_key = os.getenv("PINECONE_API_KEY")
  #print("OpenAI API key initialized successfully.")

##  The Pinecone index

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
# Retrieve the API key from environment variables
api_key = os.environ.get('PINECONE_API_KEY')
if not api_key:
    raise ValueError("PINECONE_API_KEY is not set in the environment!")

# Initialize the Pinecone client
pc = Pinecone(api_key=api_key)

In [ ]:
from pinecone import ServerlessSpec

index_name = 'genai-v1'
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
import time
import pinecone
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimension of the embedding model
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

## Querying functions

In [ ]:
def display_results(query_results):
  for match in query_results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")
    if 'metadata' in match and 'text' in match['metadata']:
        text=match['metadata']['text']
        #print(f"Text: {match['metadata']['text']}")
        target_id = query_results['matches'][0]['id']  # Get the ID from the first match
                #print(f"Target ID: {target_id}")
    else:
        print("No metadata available.")
  return text, target_id


In [ ]:
import openai
client = openai.OpenAI()
embedding_model = "text-embedding-3-small"
def get_embedding(text, model=embedding_model):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    embedding = response.data[0].embedding
    return embedding

In [ ]:
def get_query_results(query_text, namespace):
    # Generate the query vector from the query text
    query_vector = get_embedding(query_text)  # Replace with your method to generate embeddings

    # Perform the query
    query_results = index.query(
        vector=query_vector,
        namespace=namespace,
        top_k=1,  # Adjust as needed
        include_metadata=True
    )
    # Return the results
    return query_results

In [ ]:
def query_vector_store(query_text, namespace):
    print("Querying vector store...")

    # Retrieve query results
    query_results = get_query_results(query_text, namespace)

    # Process and display the results
    print("Processed query results:")
    text, target_id = display_results(query_results)

    return text, target_id

# The dataset: hotel reviews



Strategic Storytelling: Leveraging Memory in Marketing



## Download and process data

In [ ]:
download("Chapter06","hotel_reviews.csv")

In [ ]:
import pandas as pd
# Load the CSV file into a Pandas DataFrame
dfta = pd.read_csv('/content/hotel_reviews.csv',sep=',')
# display the DataFrame
dfta

### Select index

In [ ]:
index_number = 0  # Specify the index number

try:
    # Retrieve the row at the specified index
    row = dfta.iloc[index_number]

    # Extract the desired fields
    review = row['Review']
    rating = row['Rating']

    # Display the results
    print(f"Review: {review}")
    print(f"Rating: {rating}")
except IndexError:
    print(f"Error: Index {index_number} is out of bounds for the DataFrame.")
except KeyError as e:
    print(f"Error: Column '{e}' not found in the DataFrame.")

# The functions of the consumer memory agent

## Prompts and messages

In [ ]:
from reason import memory_reasoning_thread # import memory reasoning
download("commons","cot_messages_c6.py") # downloaded messages and prompts
from cot_messages_c6 import system_message_s1, generation,imcontent4,imcontent4b

## Step 1 : Memory and sentiment analysis

### The complex system message for step 1

In [ ]:
print(system_message_s1) # Print to verify

### Running the analysis

In [ ]:
# Step 1 : Memory and sentiment analysis
mrole= system_message_s1
user_text=review
retres=reason.make_openai_reasoning_call(user_text, mrole)

# Print the generated output (memory analysis)
print(retres)

## Step 2: Extract scores

In [ ]:
def extract(tasks_response):
  umessage = """
  1) Read the following text analysis that returns detailed memory tags for each part of the text
  2) Then return the list of memory tags with absolutely no other text
  3) Use no formatting, no hashtags, no markdown. Just answer in plain text
  4) Also provide the sentiment analysis score for each tag in this format(no brackets) : memory tag sentiment Score
  """
  umessage+=retres
  mrole = "system"
  mcontent = "You are a marketing expert specialized in the psychological analysis of content"
  user_role = "user"
  task_response = reason.make_openai_api_call(umessage,mrole,mcontent,user_role)
  return task_response

In [ ]:
# Step 2: Extract scores
task_response=extract(retres)
print(task_response)

## Step 3: Statistics

In [ ]:
# Step 3 : Statistics
import re

# Input text
text=task_response

# Regular expression to extract sentiment scores
pattern = r"(\d+\.\d+)"
scores = [float(match) for match in re.findall(pattern, text)]

# Output the extracted scores
print("Extracted sentiment scores:", scores)

# Optional: calculate the overall score and scaled rating
if scores:
    overall_score = sum(scores) / len(scores)
    scaled_rating = overall_score * 5
    print("Overall score (0–1):", round(overall_score, 2))
    print("Scaled rating (0–5):", round(scaled_rating, 2))

## Step 4: Content creation

### Generation Message for step 4

In [ ]:
print(generation)

### The content creation call

In [ ]:
#Step 4: Creating content
ugeneration=generation + "The advanced memory analysis of each segment of a text with a sentiment score:" + retres + " the scaled overall rating: "+ str(scaled_rating)+ " and the list of memory tags of the text "+ task_response
mrole4 = "system"
mcontent4 = imcontent4
user_role = "user"
pre_creation_response = make_openai_api_call(ugeneration,mrole4,mcontent4,user_role)
print(pre_creation_response)

In [ ]:
  umessage4b="Clean and simplify the following text for use as a DALL-E prompt. Focus on converting the detailed analysis into a concise visual description suitable for generating an engaging promotional image" + pre_creation_response
  mrole4b = "system"
  mcontent4b = imcontent4b
  user_role4b = "user"
  creation_response = make_openai_api_call(umessage4b,mrole4b,mcontent4b,user_role4b)
print(creation_response)

## Step 5 Image creation

In [ ]:
# Step 5: Creating an image
import requests
prompt=creation_response
image_url = reason.generate_image(prompt)
save_path = "c_image.png"
image_data = requests.get(image_url).content
with open(save_path, "wb") as file:
  file.write(image_data)

## Step 6 Message creation

In [ ]:
# Step 6: Creating a customer message

umessage6 = """
1) Read the following text carefully
2) Then sum it up in a paragraphs without numbering the lines
3 They output should be a text to send to a customer
"""
umessage6b=creation_response
mrole6 = "system"
mcontent6 = "You are an expert in summarization for texts to send to a customer.Begin with Dear Customer and finish with Best regards"
user_role6b = "user"
process_response = make_openai_api_call(umessage6b,mrole6,mcontent6,user_role6b)
print(process_response)

## Display outputs

In [ ]:
import os
from IPython.display import Image, display
import textwrap
# Set the desired width for each line
line_width = 70
# Wrap the text to the specified width
wrapped_message = textwrap.fill(process_response, width=line_width)
print(wrapped_message)

# Define the image path
image_path = "/content/c_image.png"

# Check if the image file exists
if os.path.exists(image_path):
    # Display the image
    display(Image(filename=image_path))
else:
    print(f"Image file {image_path} not found.")